In [2]:
import labrad
from labrad.units import WithUnit
import time
from math import log
import matplotlib.pyplot as plt
import copy

In [3]:
import labrad
scope = labrad.connection().dsox3034a
scope.connect((0x0957, 0x17a6, 'MY54100486')) #include S/N if more than one device
scope.setAcquireType('hres')
scope.setWaveformPoints(100)

Enter username, or blank for the global user (localhost:7682): 
Enter LabRAD password (localhost:7682): ········


In [4]:
cxn = labrad.connect()
pulser = cxn.pulser

In [5]:
def pON(n):
    return pulser.output('DDS_' + str(n), True)

def pOFF(n):
    return pulser.output('DDS_' + str(n), False)

In [5]:
pulser.amplitude('DDS_0', WithUnit(10, 'dBm'))
pulser.frequency('DDS_0', WithUnit(50, 'MHz'))

Value(50.0, 'MHz')

In [38]:
#delays determined by characterizing equipment
global_delay = 0.375
abs_phase = 0   
f_delay = 0.2
p_delay = 0.500     #0.218 + 0.14
a_delay = 0.2
total_delay = f_delay + p_delay + a_delay
f_delay_u = WithUnit(f_delay, 'us')
p_delay_u = WithUnit(p_delay, 'us')
a_delay_u = WithUnit(a_delay, 'us')
off_amp = WithUnit(-63, 'dBm')

def split(DDS_sequence): 
    off_amp = WithUnit(-63, 'dB')
    new_sequence = [] 
    for dds in DDS_sequence:
        start = dds[1][dds[1].units] #us
        duration = dds[2][dds[2].units] #us
        p_start_u = WithUnit(start - f_delay - p_delay - a_delay, 'us')
        f_start_u = WithUnit(start - f_delay - a_delay, 'us')
        a_start_u = WithUnit(start - a_delay, 'us')
        dds_phase = [(dds[0], p_start_u, p_delay_u, dds[3], WithUnit(-63, 'dBm'), dds[5], dds[6], dds[7])]
        dds_freq = [(dds[0], f_start_u, f_delay_u, dds[3], WithUnit(-63, 'dBm'), dds[5], dds[6], dds[7])]
        dds_amp = [(dds[0], a_start_u, WithUnit(a_delay + duration, 'us'), dds[3], dds[4], dds[5], dds[6], dds[7])]
        new_sequence += dds_phase
        new_sequence += dds_freq
        new_sequence += dds_amp
    return new_sequence
   
def check(DDS_sequence): 
    for n in range(0, len(DDS_sequence)-1):
        curr_end = DDS_sequence[n][1][DDS_sequence[n][1].units] + DDS_sequence[n][2][DDS_sequence[n][2].units]
        next_start = DDS_sequence[n+1][1][DDS_sequence[n+1][1].units]
        if (next_start - curr_end <= total_delay): 
            raise ValueException("Time between pulse ", n, " and pulse ", n+1, " is too short by ", abs(next_start - curr_end), ".")      

def phase_coherent1(DDS_sequence): 
    raw_pulses = [[]]
    new_pulses = [[]]
    for n in range(len(DDS_sequence)):
        dds = DDS_sequence[n]
        frequency = dds[3][dds[3].units]
        start = dds[1][dds[1].units]
        duration = dds[2][dds[2].units]
        end = start + duration
        pulse = []
        pulse.append(frequency)
        pulse.append(start)
        pulse.append(duration) 
        pulse.append(end)
        pulse.append(0) #phase
        pulse.append(0) 
        raw_pulses.append(pulse)
    raw_pulses = raw_pulses[1:]
    new_pulses[0] = raw_pulses[0]
    for m in range(1, len(raw_pulses)): 
        curr = raw_pulses[m]
        last = raw_pulses[m-1]
        gap = curr[1] - last[3]
        if gap != 0: 
            zero_pulse = [last[0], last[3], gap, curr[1], 0,  1]
            new_pulses.append(zero_pulse)
        new_pulses.append(curr)
    print "RAW PULSES"
    for p in raw_pulses: 
        print p
    return new_pulses

def phase_coherent2(pulses): 
    abs_phase = 0
    ftp = {}
    r_phase = 0
    for n in range(len(pulses)):
        freq = pulses[n][0]
        start = pulses[n][1]
        duration = pulses[n][2]
        end = pulses[n][3]
        phase = pulses[n][4]
        is_zero = pulses[n][5]
        if freq not in ftp:
            abs_phase = compute_phase(abs_phase, duration, freq)
            ftp[freq] = abs_phase
            pulses[n][4] = 0
            #print "New frequency: ", freq
            #print "Phase shift: ", 0
            #print "Phase at end of pulse: ", abs_phase
        else: 
            if is_zero: 
                abs_phase = compute_phase(abs_phase, duration, freq)
                #print "Zero pulse: ", freq
                #print "Phase shift: ", 0
                #print "Phase at end of pulse: ", abs_phase
            else: 
                #print "Repeat pulse: ", freq
                last_phase = ftp[freq]
                #print "Last phase for freq: ", last_phase
                #print "Phase at start of this pulse: ", abs_phase
                r_phase = rel_phase(compute_phase(abs_phase, amp_delay, freq), last_phase)
                abs_phase = (abs_phase + r_phase)%360
                ftp[freq] = abs_phase
                pulses[n][4] = r_phase
                abs_phase = compute_phase(abs_phase, duration, freq)
                ftp[freq] = abs_phase
                #print "Phase shift: ", r_phase
                #print "Phase at end of pulse: ", abs_phase
    print "PULSE TABLE"
    for p in pulses: 
        print p
    return pulses

def phase_coherent3(pulses, DDS_sequence): 
    new_sequence= []
    n = 0
    for pulse in pulses: 
        if pulse[5]:
            continue
        else: 
            new_dds = [(copy.deepcopy(DDS_sequence[n][0]), copy.deepcopy(DDS_sequence[n][1]), 
                        copy.deepcopy(DDS_sequence[n][2]), copy.deepcopy(DDS_sequence[n][3]), 
                        copy.deepcopy(DDS_sequence[n][4]), WithUnit(pulse[4], 'deg'), #use this as adjustment parameter? 
                        copy.deepcopy(DDS_sequence[n][6]), copy.deepcopy(DDS_sequence[n][7]))]
            new_sequence += new_dds #deep copy error
            n+=1
    print "FINAL SEQUENCE"
    for s in new_sequence: 
        print s
    return new_sequence

def phase_coherent(DDS_sequence): 
    pulse_table = phase_coherent1(DDS_sequence)
    phase_updated_pulse_table = phase_coherent2(pulse_table)
    return phase_coherent3(phase_updated_pulse_table, DDS_sequence)

def rel_phase(p1, p2): 
    diff = p1-p2
    if diff < 0: 
        return (diff+360)%360
    else: 
        return diff%360

def compute_phase(initial_phase, dt, frequency): 
    phase = (dt*frequency*360)%360
    return (initial_phase + phase)%360
                            
def process_sequence(dds_sequence): 
    return phase_coherent(split(dds_sequence))


In [39]:
amp = WithUnit(8, 'dBm')
amp1 = WithUnit(15, 'dBm')
phase = WithUnit(0,'deg')
flip_phase = WithUnit(180,'deg')
amp_ramp_rate = WithUnit(0,'dB')
freq_ramp_rate = WithUnit(0,'MHz')

### DEFINE DDS SEQUENCE ###
t3 = [
#('DDS_1', WithUnit(5,'us'), WithUnit(110.00,  'us'), WithUnit(80.0,   'MHz'), amp, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(10,'us'), WithUnit(10.00,  'us'), WithUnit(100.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(40,'us'), WithUnit(10.00,  'us'), WithUnit(50.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(70,'us'), WithUnit(10.00,  'us'), WithUnit(100.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
('DDS_2', WithUnit(100,'us'), WithUnit(10.00,  'us'), WithUnit(50.0,   'MHz'), amp1, phase, freq_ramp_rate, amp_ramp_rate),
]

### DEFINE TTL SEQUENCE ###
t3_switch_ttl = [(10,10), (40,10), (70,10), (100, 10)]

### RUN PROCEDURE ###
print "Starting sequence..."

scope.single()
time.sleep(1)
pulser.amplitude('DDS_1', amp)
pulser.frequency('DDS_1', WithUnit(100.0,   'MHz'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
pulser.new_sequence()
        
for ttl in t3_switch_ttl: 
    pulser.add_ttl_pulse('ttl_3',WithUnit(ttl[0],'us'),WithUnit(ttl[1],'us'))
        
pulser.add_dds_pulses(process_sequence(t3))
pulser.program_sequence()
pulser.start_number(1)
pulser.wait_sequence_done()
pulser.stop_sequence()
pulser.amplitude('DDS_1',WithUnit(-63,'dBm'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
time.sleep(1)
print "Sequence done."

Starting sequence...
RAW PULSES
[100.0, 9.403000000000002, 0.197, 9.600000000000001, 0, 0]
[100.0, 9.600000000000001, 0.2, 9.8, 0, 0]
[100.0, 9.8, 10.2, 20.0, 0, 0]
[50.0, 39.40299999999999, 0.197, 39.599999999999994, 0, 0]
[50.0, 39.599999999999994, 0.2, 39.8, 0, 0]
[50.0, 39.8, 10.2, 50.0, 0, 0]
[100.0, 69.40299999999999, 0.197, 69.6, 0, 0]
[100.0, 69.6, 0.2, 69.8, 0, 0]
[100.0, 69.8, 10.2, 80.0, 0, 0]
[50.0, 99.40299999999999, 0.197, 99.6, 0, 0]
[50.0, 99.6, 0.2, 99.8, 0, 0]
[50.0, 99.8, 10.2, 110.0, 0, 0]
PULSE TABLE
[100.0, 9.403000000000002, 0.197, 9.600000000000001, 0, 0]
[100.0, 9.600000000000001, 0.2, 9.8, 0.0, 0]
[100.0, 9.8, 10.2, 20.0, 0.0, 0]
[100.0, 20.0, 19.40299999999999, 39.40299999999999, 0, 1]
[50.0, 39.40299999999999, 0.197, 39.599999999999994, 0, 0]
[50.0, 39.599999999999994, 0.2, 39.8, 0.0, 0]
[50.0, 39.8, 10.2, 50.0, 0.0, 0]
[50.0, 50.0, 19.40299999999999, 69.40299999999999, 0, 1]
[100.0, 69.40299999999999, 0.197, 69.6, 107.99999999962165, 0]
[100.0, 69.6, 0.2, 6

In [ ]:
scope.single()

In [150]:
### RUN PROCEDURE ###
print "Starting sequence..."

scope.single()
time.sleep(1)
pulser.amplitude('DDS_1',WithUnit(-63,'dBm'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
pulser.new_sequence()
        
for ttl in t3_switch_ttl: 
    pulser.add_ttl_pulse('ttl_3',WithUnit(ttl[0],'us'),WithUnit(ttl[1],'us'))
        
pulser.add_dds_pulses(t3)
pulser.program_sequence()
pulser.start_number(1)
pulser.wait_sequence_done()
pulser.stop_sequence()
pulser.amplitude('DDS_1',WithUnit(-63,'dBm'))
pulser.amplitude('DDS_2',WithUnit(-63,'dBm'))
time.sleep(1)
print "Sequence done."

Starting sequence...
Sequence done.


In [243]:
pulser.amplitude('DDS_1',WithUnit(5,'dBm'))
pulser.amplitude('DDS_2',WithUnit(10,'dBm'))
pulser.frequency('DDS_1',WithUnit(60,'MHz'))
pulser.frequency('DDS_2',WithUnit(60,'MHz'))

Value(60.0, 'MHz')

In [318]:
pulser

LabRAD Server: Pulser (ID=4)




Settings:
    add_dds_pulses
    add_ttl_pulse
    add_ttl_pulses
    amplitude
    clear_dds_lock
    complete_infinite_iteration
    debug
    echo
    extend_sequence_length
    frequency
    get_channels
    get_collection_mode
    get_collection_time
    get_dds_amplitude_range
    get_dds_channels
    get_dds_frequency_range
    get_line_trigger_limits
    get_pmt_counts
    get_readout_counts
    get_secondary_pmt_counts
    get_state
    get_timetag_resolution
    get_timetags
    human_readable_dds
    human_readable_ttl
    internal_advance_dds
    internal_reset_dds
    line_trigger_duration
    line_trigger_state
    new_sequence
    output
    program_sequence
    reinitialize_dds
    repeatitions_completed
    reset_fifo_normal
    reset_readout_counts
    reset_timetags
    set_collection_time
    set_mode
    signal__log
    signal__new_dds_parameter
    signal__new_line_trigger_parameter
    signal__switch_toggled
    start_infinite
   